<a href="https://colab.research.google.com/github/JoaoPauloRadd/BERT_ADAMW/blob/main/BERT_fb_yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Visando o trabalho da disciplina, realizarei aqui o teste com um modelo BART pré treinado com o Yahoo respostas voltado para classificações de rótulos variados à escolha.

**Descrição do Modelo**

Este modelo pega facebook / bart-large-mnli e o ajusta na classificação de tópicos do Yahoo Answers. Ele pode ser usado para prever se um rótulo de tópico pode ser atribuído a uma determinada sequência, independentemente de o rótulo ter sido visto antes ou não.

Link para mais detalhes https://huggingface.co/joeddav/bart-large-mnli-yahoo-answers

In [ ]:
pip install transformers

     |████████████████████████████████| 1.9MB 16.4MB/s 
     |████████████████████████████████| 3.2MB 52.4MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=3334b36fcb34d837d7956a6d3cbed13ca55a8df5819e4759aa90cf90c4db16f1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BartTokenizerFast, BartForSequenceClassification

tokenizer = BartTokenizerFast.from_pretrained("joeddav/bart-large-mnli-yahoo-answers",add_prefix_space=True)

model = BartForSequenceClassification.from_pretrained("joeddav/bart-large-mnli-yahoo-answers")

Com isso serão carregados os modelos. O modelo pode ser usado com o zero-shot-classificationpipeline como segue.

In [ ]:
from transformers import pipeline
nlp = pipeline("zero-shot-classification", model="joeddav/bart-large-mnli-yahoo-answers")

sequence_to_classify = "Who are you voting for in 2020?"
candidate_labels = ["Europe", "public health", "politics", "elections"]
hypothesis_template = "This text is about {}."
nlp(sequence_to_classify, candidate_labels, multi_class=True, hypothesis_template=hypothesis_template)

{'labels': ['politics', 'elections', 'Europe', 'public health'],
 'scores': [0.9845868945121765,
  0.9786983132362366,
  0.37489765882492065,
  0.02557758055627346],
 'sequence': 'Who are you voting for in 2020?'}

Resultado esperado:


```
{'labels': ['politics', 'elections', 'Europe', 'public health'],
 'scores': [0.9845868945121765,
  0.9786983132362366,
  0.37489765882492065,
  0.02557758055627346],
 'sequence': 'Who are you voting for in 2020?'}
```



[texto do link](https://)Agora farei um teste utilizando algumas classes do meu artigo base relativas a especificidade do argumento. A sequencia para a classificação será uma da própria base. (Observando que esse BERT não foi treinado nesse conjunto)



In [ ]:
sequence_to_classify = "Okay. I said, 'Neely is upset about his career being over and he does not like to think about his past because of his injury.' "
candidate_labels = ["Low", "Medium", "High"]
hypothesis_template = "This text is about {}."
nlp(sequence_to_classify, candidate_labels, multi_class=False, hypothesis_template=hypothesis_template)

{'labels': ['Low', 'High', 'Medium'],
 'scores': [0.3847925662994385, 0.32908764481544495, 0.2861197888851166],
 'sequence': "Okay. I said, 'Neely is upset about his career being over and he does not like to think about his past because of his injury.' "}

Era esperado "Medium", mas retorna "Low":


```
{'labels': ['Low', 'High', 'Medium'],
 'scores': [0.3847925662994385, 0.32908764481544495, 0.2861197888851166],
 'sequence': "Okay. I said, 'Neely is upset about his career being over and he does not like to think about his past because of his injury.' "}
```



In [ ]:
sequence_to_classify = "And in the book, it says, 'I don't wanna talk about football. I don't want to hear how great I used to be, how great I was,' on page 71. And Flick, it didn't really talk about how he felt. It only talked about his career after and how he wasn't as good anymore."
candidate_labels = ["Low", "Medium", "High"]
hypothesis_template = "This text is about {}."
nlp(sequence_to_classify, candidate_labels, multi_class=False, hypothesis_template=hypothesis_template)

{'labels': ['High', 'Low', 'Medium'],
 'scores': [0.42077988386154175, 0.3369370400905609, 0.24228313565254211],
 'sequence': "And in the book, it says, 'I don't wanna talk about football. I don't want to hear how great I used to be, how great I was,' on page 71. And Flick, it didn't really talk about how he felt. It only talked about his career after and how he wasn't as good anymore."}

Nesse casso é alcançado "High" como esperado:


```
{'labels': ['High', 'Low', 'Medium'],
 'scores': [0.42077988386154175, 0.3369370400905609, 0.24228313565254211],
 'sequence': "And in the book, it says, 'I don't wanna talk about football. I don't want to hear how great I used to be, how great I was,' on page 71. And Flick, it didn't really talk about how he felt. It only talked about his career after and how he wasn't as good anymore."}
```



Este modelo é somente pré treinado em um conjunto diferente e pode ser treinado no conjunto de estudo. O experimento em que é executado é feito uso de GloVe e uma rede neural pré treinada em outra base semelhante. acredito que é posívem fazer nocas combinações para melhorar os resultados finais em um experimento.

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('GPU encontrada em: {}'.format(device_name))
else:
  raise SyntaxError('GPU não encontrada')

GPU encontrada em: /device:GPU:0


In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import os
import pandas as pd

path = "/content/drive/MyDrive/DTC/Discussion Tracker Corpus/DT.2 (with corrections)"
files = os.listdir(path)
files


['T124.EAGER.1.Ivan.Final.xlsx',
 'T124.EAGER.2.Night.Final.xlsx',
 'T124.EAGER.3.TheName.Final.xlsx',
 'T125.EAGER.1.Sleepyhollow.Final.xlsx',
 'T125.EAGER.2.Ministersveil.Final.xlsx',
 'T125.EAGER.3.Poems.Final.xlsx',
 'T126.EAGER.1.Lordflies.Final.xlsx',
 'T126.EAGER.2.Mockingbird.Final.xlsx',
 'T126.EAGER.3.Mockingbird.Final.xlsx',
 'T127.EAGER.1.Heartdark.Final.xlsx',
 'T127.EAGER.2.Scarlet.Final.xlsx',
 'T127.EAGER.3.Shakespeare.Final.xlsx',
 'T128.EAGER.1.Mockingbird.Final.xlsx',
 'T128.EAGER.2.Wicked.Final.xlsx',
 'T128.EAGER.3.Littleprince.Final.xlsx',
 'T130.EAGER.1.OfMice.Final.xlsx',
 'T129.EAGER.1.Immortallife.Final.xlsx',
 'T129.EAGER.2.Crucible.Final.xlsx',
 'T129.EAGER.3.Wild.Final.xlsx',
 'T130.EAGER.2.Fahrenheit.Final.xlsx',
 'T130.EAGER.3.King.Final.xlsx',
 'T131.EAGER.2.YellowWall.Final.xlsx',
 'T131.EAGER.3.Antigone.Final.xlsx',
 'T132.EAGER.1.Witches.Final.xlsx',
 'T132.EAGER.2.Crucible.Final.xlsx',
 'T133.EAGER.1.Bleachers.Final.xlsx',
 'T133.EAGER.2.JFK.Final.xl

In [ ]:
df = pd.DataFrame()
for f in files:
    data = pd.read_excel(path+"/"+f)
    df = df.append(data)

In [ ]:
df

,Disc id,Sp id,Talk,Topic,Collaboration Code,Turn of Reference,Argument Segmentation,Claim,Evidence,Warrant,Textual,Intertextual,Experiential,Part,Detail,Content,Chain,Low,Med,High,Factual/Literal,Interpretive,Unnamed: 20,Unnamed: 21,Factual,Interp
0,T124EAGER1.IVAN.1,T124,"See all your smiling faces. Okay, St. ? xx xx....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pip install nlp

     |████████████████████████████████| 1.7MB 18.3MB/s 
     |████████████████████████████████| 245kB 54.9MB/s 


In [ ]:
!pip install datasets

     |████████████████████████████████| 184kB 15.9MB/s 
     |████████████████████████████████| 112kB 16.3MB/s 


Base carregada, agora é limpar

In [ ]:
from datasets import Dataset
import numpy as np


df = df.applymap(str)
df = df.replace("nan", np.nan)
df = df.replace({"Collaboration Code": "Non"}, {"Collaboration Code": 0})
df = df.replace({"Collaboration Code": "E"}, {"Collaboration Code": 1})
df = df.replace({"Collaboration Code": "e"}, {"Collaboration Code": 1})
df = df.replace({"Collaboration Code": "N"}, {"Collaboration Code": 2})
df = df.replace({"Collaboration Code": " N"}, {"Collaboration Code": 2})
df = df.replace({"Collaboration Code": "C"}, {"Collaboration Code": 3})
df = df.replace({"Collaboration Code": "A"}, {"Collaboration Code": 4})



dataCol = df[["Disc id","Talk","Collaboration Code"]]
dataCol = dataCol.dropna()
dataCol = dataCol.reset_index(drop=True)
dataCol

,Disc id,Talk,Collaboration Code
0,T124EAGER1.IVAN.3,Um I thought-,0.0
1,T124EAGER1.IVAN.5,Um. I thought that uh the beginning of the nov...,2.0
2,T124EAGER1.IVAN.7,{overlapping} Yeah. Yeah,0.0
3,T124EAGER1.IVAN.9,"Um I was just going to say, like, it was to sh...",1.0
4,T124EAGER1.IVAN.11,"Like, maybe, like, he didn't have a positive e...",1.0
...,...,...,...
3258,T132.EAGER.3.Wallflower.trimmed.260,Well it just depends.,2.0
3259,T132.EAGER.3.Wallflower.trimmed.262,How you look at it.,1.0
3260,T132.EAGER.3.Wallflower.trimmed.264,Go ahead.,0.0
3261,T132.EAGER.3.Wallflower.trimmed.265,I think maybe Charlie and Sam because he knows...,1.0


Agora vamos treinar com a base

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.preprocessing.sequence import pad_sequences

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#Para calcular métricas adicionais além da perda, você também pode definir sua própria compute_metricsfunção e passá-la para o treinador.
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
def tokenize(batch):
    return tokenizer(batch['label'],is_split_into_words=True, padding=True, truncation=True)
def encodeColumn(col):
    arr = []
    for sent in col:
        # `encode` vai:
        #   (1) Tokenizar a sentença.
        #   (2) Criar os tokens especiais no inicio e fim da sentença (`[CLS]` `[SEP]`).
        #   (3) Mapeia os tokens à suas IDs.
        encoded_sent = tokenizer.encode(
                            sent,
                            add_special_tokens = True,
                      )

        # Adiciona à lista a sentença que foi encoded
        arr.append(encoded_sent)
    return arr

# Create sentence and label lists
dataCol=dataCol.rename(columns={'Collaboration Code':'label'})
sentences = dataCol.Talk.values
labels = dataCol.label.values.astype(np.float32)


# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )

    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=1077,
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)





In [ ]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split
# Use 80% for training and 20% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.2)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# The DataLoader needs to know our batch size for training, so we specify it
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.
batch_size = 32
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BartTokenizerFast, BartForSequenceClassification, AdamW, BartConfig

model = BartForSequenceClassification.from_pretrained(
    "joeddav/bart-large-mnli-yahoo-answers", #
    num_labels = 3, #
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.save_pretrained('./saved_model/')

# Tell pytorch to run this model on the GPU.
model.cuda()



RuntimeError: ignored

In [ ]:









# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
# Set the batch size.
batch_size = 32
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=None)

######
model = model.to(device)

# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables
predictions , true_labels = [], []
# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')

NameError: ignored

------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#Para calcular métricas adicionais além da perda, você também pode definir sua própria compute_metricsfunção e passá-la para o treinador.
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
def tokenize(batch):
    return tokenizer(batch['label'],is_split_into_words=True, padding=True, truncation=True)
def encodeColumn(col):
    input_ids = []
    for sent in col:
        # `encode` vai:
        #   (1) Tokenizar a sentença.
        #   (2) Criar os tokens especiais no inicio e fim da sentença (`[CLS]` `[SEP]`).
        #   (3) Mapeia os tokens à suas IDs.
        encoded_sent = tokenizer.encode(
                            sent,
                            add_special_tokens = True,
                      )

        # Adiciona à lista a sentença que foi encoded
        input_ids.append(encoded_sent)
    return input_ids
#preparadados para classificar colaboração
#dados=dataCol[["Disc id","Talk"]]
#Y=dataCol["Collaboration Code"]
#xTreinamento, xTeste, yTreinamento, yTeste = train_test_split(dados, Y, test_size=0.33)


dataCol=dataCol.rename(columns={'Collaboration Code':'label'})
#dataCol['label'] = encodeColumn(dataCol['label'])

test_dataset, train_dataset = train_test_split(dataCol,test_size=0.33,shuffle=True)#
train_dataset = train_dataset.reset_index(drop=True)
test_dataset = test_dataset.reset_index(drop=True)


#test_dataset['label'] = encodeColumn(test_dataset['label'])

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)


In [ ]:
test_dataset['label']


In [ ]:


train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

#train_dataset['label'] = encodeCol(train_dataset['label'])
#test_dataset['label'] = encodeCol(test_dataset['label'])

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'],maxlen=1077 )
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'],maxlen=1077 )
#train_dataset=xTreinamento
#train_dataset["Collaboration Code"]=yTreinamento
#test_dataset=xTeste
#test_dataset["Collaboration Code"]=yTeste

#prepara parâmetros
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainerC = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)


AssertionError: ignored

In [ ]:
train_dataset['label']

TypeError: ignored

In [ ]:
trainerC.train()

TypeError: ignored

In [ ]:
trainer.evaluate()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs